In [1]:
from torch_geometric.nn import GCNConv, SplineConv
from torch_geometric import seed_everything
import torch
import torch_geometric as pyg
import numpy as np

pos = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).astype(np.float32)
x = np.array([1, 2, 3, 4]).reshape(-1, 1).astype(np.float32)
edge_index = np.array([[0, 1], [1, 2], [2, 3], [3, 0]]).T

pos = torch.tensor(pos)
x = torch.tensor(x)
edge_index = torch.tensor(edge_index)
data = pyg.data.Data(x, edge_index, pos = pos)

data = pyg.transforms.Distance()(data)
data

Data(x=[4, 1], edge_index=[2, 4], pos=[4, 2], edge_attr=[4, 1])

### GCNs are rotation invariant

In [2]:
def rotate_graph(data, theta):
    theta = np.radians(theta)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    
    rot_pos = torch.tensor((R@pos.numpy().T).T.astype(np.float32))

    rotated = pyg.data.Data(data.x, data.edge_index, pos = rot_pos)
    if data.edge_attr.shape[-1] == 1:
        rotated = pyg.transforms.Distance()(rotated)
    else: 
        rotated = pyg.transforms.Cartesian(norm=True, cat=False)(rotated)
    return rotated

In [3]:
seed_everything(0)
gc = GCNConv(1, 1)
gc(data.x, data.edge_index, data.edge_attr)

tensor([[2.4760],
        [1.4441],
        [2.3994],
        [3.2438]], grad_fn=<AddBackward0>)

In [4]:
rotated = rotate_graph(data, 30)
gc(rotated.x, rotated.edge_index, rotated.edge_attr)

tensor([[2.4760],
        [1.4441],
        [2.3994],
        [3.2438]], grad_fn=<AddBackward0>)

In [5]:
data = pyg.transforms.Cartesian(norm=True, cat=False)(data)

In [6]:
seed_everything(0)
sc = SplineConv(1, 1, 2, 5)

sc(data.x, data.edge_index, data.edge_attr)

/home/hussain/miniconda3/envs/tactile38/lib/python3.8/site-packages/torch_geometric/nn/conv/spline_conv.py:133: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  warnings.warn(


tensor([[0.9901],
        [1.6638],
        [3.0486],
        [3.1803]], grad_fn=<AddBackward0>)

In [7]:
rotated = rotate_graph(data, 30)
sc(rotated.x, rotated.edge_index, rotated.edge_attr)

tensor([[1.1142],
        [1.8102],
        [2.7647],
        [3.6195]], grad_fn=<AddBackward0>)